In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage, HumanMessage
from langchain.agents.middleware import ModelRequest, ModelResponse, wrap_model_call

In [ ]:
basic_model = init_chat_model('gpt-3.5-turbo')
advanced_model = init_chat_model('gpt-4.1-mini')

In [ ]:
@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    message_count = len(request.state.messages)
    
    if message_count > 3:
        model = advanced_model
    else:
        model = basic_model
    
    request.model = model
    
    return handler(request)

In [ ]:
agent = create_agent(
    model=basic_model,
    middlewares=[dynamic_model_selection]
)

In [ ]:
response = agent.invoke(
    {
        'messages': 
            [
                SystemMessage(content='You are a helpful assistant.'),
                HumanMessage(content='Tell me a joke.'),   
            ]
    }
)

In [ ]:
print(response['message'][-1]['content'])
print(response['message'][-1]['responce_metadata']['model_name'])